# Processamento de Dados em Larga Escala com Spark

## Informação Básica
- **Título do Projeto**: Processamento de Dados em Larga Escala com Spark
- **Alunos**:
  - João Carneiro, Nº 50938
  - Eduardo Abrantes, Nº 50391

## Contribuição

| Aluno        | Tarefa realizada                                                                 | Horas estimadas |
|--------------|----------------------------------------------------------------------------------|-----------------|
| João Carneiro   | ...  | 7h             |
| Eduardo Abrantes  | ...        | 7h             |

## Background e Motivação

- Este projeto tem como objetivo ...

# Pré-Requisitos

## Instalação dos Dados
Antes de iniciar o projeto, é necessário realizar o download e preparação dos Dados Meteorológicos do GHCN-Daily. Siga os passos abaixo:

## Setup de Dependências
Para a realização deste projeto, foram utilizadas bibliotecas fundamentais do Python para Ciência de Dados. A instalação das dependências foi realizada através do comando:

In [1]:
!pip install wikiextractor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.1 MB/s eta 0:00:00


#### Descrição das Bibliotecas

- wikiextractor: ...

- ...

## Extração do Dump da Wikipedia

...

O seguinte código foi utilizado para esta operação inicial:

In [15]:
!ls -lh /home/jovyan/work/proj-three-data-science/data

total 23G
-rwxrwxrwx 1 root   root   23G May 28 14:52 enwiki-latest-pages-articles.xml.bz2
drwxr-xr-x 1 jovyan users 4.0K May 28 16:34 wikipedia-dump


In [ ]:
!python -m wikiextractor.WikiExtractor "/home/jovyan/work/proj-three-data-science/data/enwiki-latest-pages-articles.xml.bz2" -o "/home/jovyan/work/proj-three-data-science/data/wikipedia-dump/text" --no-templates

INFO: Starting page extraction from /home/jovyan/work/proj-three-data-science/data/enwiki-latest-pages-articles.xml.bz2.
INFO: Using 11 extract processes.
INFO: Extracted 100000 articles (490.7 art/s)
INFO: Extracted 200000 articles (710.0 art/s)
INFO: Extracted 300000 articles (974.1 art/s)
INFO: Extracted 400000 articles (1204.7 art/s)
INFO: Extracted 500000 articles (1423.6 art/s)
INFO: Extracted 600000 articles (1102.4 art/s)
INFO: Extracted 700000 articles (1176.8 art/s)
INFO: Extracted 800000 articles (1220.5 art/s)
INFO: Extracted 900000 articles (1328.8 art/s)
INFO: Extracted 1000000 articles (1374.7 art/s)
INFO: Extracted 1100000 articles (1668.2 art/s)
INFO: Extracted 1200000 articles (1697.4 art/s)
INFO: Extracted 1300000 articles (1600.3 art/s)
INFO: Extracted 1400000 articles (1699.1 art/s)
INFO: Extracted 1500000 articles (1713.2 art/s)
INFO: Extracted 1600000 articles (1406.2 art/s)
INFO: Extracted 1700000 articles (1749.1 art/s)
INFO: Extracted 1800000 articles (1797.5 

# Exercício 1 ...

# Declaração de Integridade

> Eu, João Carneiro, estudante com o número de inscrição 50938 do 1º Ciclo em Informática Web, Móvel e na Nuvem da Universidade da Beira Interior, declaro ter desenvolvido o presente trabalho e elaborado o presente texto em total consonância com o Código de Integridade da Universidade da Beira Interior. Mais concretamente afirmo não ter incorrido em qualquer das variedades de Fraude Académica, e que aqui declaro conhecer, que em particular atendi à exigida referenciação de frases, extratos, imagens e outras formas de trabalho intelectual, e assumindo assim na íntegra as responsabilidades da autoria


> Eu, Eduardo Abrantes, estudante com o número de inscrição 50391 do 1º Ciclo em Informática Web, Móvel e na Nuvem da Universidade da Beira Interior, declaro ter desenvolvido o presente trabalho e elaborado o presente texto em total consonância com o Código de Integridade da Universidade da Beira Interior. Mais concretamente afirmo não ter incorrido em qualquer das variedades de Fraude Académica, e que aqui declaro conhecer, que em particular atendi à exigida referenciação de frases, extratos, imagens e outras formas de trabalho intelectual, e assumindo assim na íntegra as responsabilidades da autoria

Universidade da Beira Interior, Covilhã  
02/06/2025
